In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import Row

jar_files = [
    "/util/clickhouse-jdbc-0.3.2-all.jar"
]

# Initialize Spark session with JARs
spark = SparkSession.builder \
    .appName("test-jdbc") \
    .master("spark://spark-master:7077") \
    .config("spark.jars", ",".join(jar_files)) \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.sql.extensions","io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog","org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.hadoop.fs.s3a.access.key","NzUmXT2E7YJIY3m9yFBz") \
    .config("spark.hadoop.fs.s3a.secret.key","5SCMGEtt52EziOcTUi9dyMt5dfFE2MBaK1ZaskPX") \
    .config("spark.hadoop.fs.s3a.endpoint","http://minio:9000") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .getOrCreate()


url = "jdbc:ch://clickhouse:8123/cameraaberta"
user = "admin" 
password = "admin"  
driver = "com.clickhouse.jdbc.ClickHouseDriver"

display(spark)

In [2]:
#stops

df = spark.read.option("inferSchema", True).option("header", "true") \
    .csv(f"s3a://raw/stops.txt")
# show data
df.show(10)

df.printSchema()

# Write DataFrame to ClickHouse
df.write \
    .format("jdbc") \
    .option("driver", driver) \
    .option("url", url) \
    .option("user", user) \
    .option("password", password) \
    .option("dbtable", "stops") \
    .option("createTableOptions", "ENGINE=MergeTree() ORDER BY (stop_id)") \
    .mode("overwrite") \
    .save()



+-------+---------------+---------+----------+----------+
|stop_id|      stop_name|stop_desc|  stop_lat|  stop_lon|
+-------+---------------+---------+----------+----------+
|  18848|       Clínicas|     null|-23.554022|-46.671108|
|  18849|  Vila Madalena|     null|-23.546498|-46.691141|
|  18850|     Consolação|     null|-23.558094|-46.660205|
|  18851|      Conceição|     null|-23.635039|-46.641239|
|  18852|      Jabaquara|     null|-23.646033|-46.641028|
|  18853|      São Judas|     null|-23.625882|-46.640936|
|  18854|          Saúde|     null|-23.618245|-46.639139|
|  18855|Praça Da Árvore|     null|-23.610583|-46.637918|
|  18856|     Santa Cruz|     null|-23.598541|-46.636638|
|  18857|   Vila Mariana|     null|-23.589359|-46.634677|
+-------+---------------+---------+----------+----------+
only showing top 10 rows

root
 |-- stop_id: integer (nullable = true)
 |-- stop_name: string (nullable = true)
 |-- stop_desc: string (nullable = true)
 |-- stop_lat: double (nullable = t

In [3]:
#trips

df = spark.read.option("inferSchema", True).option("header", "true") \
    .csv(f"s3a://raw/trips.txt")
# show data
df.show(10)

df.printSchema()

# Write DataFrame to ClickHouse
df.write \
    .format("jdbc") \
    .option("driver", driver) \
    .option("url", url) \
    .option("user", user) \
    .option("password", password) \
    .option("dbtable", "trips") \
    .option("createTableOptions", "ENGINE=MergeTree() ORDER BY (route_id)") \
    .mode("overwrite") \
    .save()


+--------+----------+---------+--------------------+------------+--------+
|route_id|service_id|  trip_id|       trip_headsign|direction_id|shape_id|
+--------+----------+---------+--------------------+------------+--------+
| 1012-10|       USD|1012-10-0|      Jd. Monte Belo|           0|   81072|
| 1012-10|       USD|1012-10-1|  Term. Jd. Britania|           1|   81073|
| 1012-21|       U__|1012-21-0|         Jd. Rosinha|           0|   81195|
| 1014-10|       U__|1014-10-0|          Cptm Perus|           0|   82855|
| 1015-10|       USD|1015-10-0|Chác. Maria Trindade|           0|   81148|
| 1016-10|       USD|1016-10-0|  Shop. Center Norte|           0|   72283|
| 1016-10|       USD|1016-10-1|       Cem. Do Horto|           1|   82172|
| 1017-10|       USD|1017-10-0|   Conexão Vl. Iório|           0|   72355|
| 1017-10|       USD|1017-10-1|               Perus|           1|   72356|
| 1018-10|       USD|1018-10-0|       Metrô Santana|           0|   71029|
+--------+----------+----

In [4]:
#stop-times

df = spark.read.option("inferSchema", True).option("header", "true") \
    .csv(f"s3a://raw/stop_times.txt")
# show data
df.show(10)

df.printSchema()

# Write DataFrame to ClickHouse
df.write \
    .format("jdbc") \
    .option("driver", driver) \
    .option("url", url) \
    .option("user", user) \
    .option("password", password) \
    .option("dbtable", "stop_times") \
    .option("createTableOptions", "ENGINE=MergeTree() ORDER BY (stop_id, stop_sequence)") \
    .mode("overwrite") \
    .save()

+---------+-------------------+-------------------+--------+-------------+
|  trip_id|       arrival_time|     departure_time| stop_id|stop_sequence|
+---------+-------------------+-------------------+--------+-------------+
|1012-10-0|2025-02-04 18:00:00|2025-02-04 18:00:00|  301790|            1|
|1012-10-0|2025-02-04 18:01:35|2025-02-04 18:01:35|  301764|            2|
|1012-10-0|2025-02-04 18:03:10|2025-02-04 18:03:10|  301724|            3|
|1012-10-0|2025-02-04 18:04:45|2025-02-04 18:04:45|  301730|            4|
|1012-10-0|2025-02-04 18:06:20|2025-02-04 18:06:20|30003042|            5|
|1012-10-0|2025-02-04 18:07:55|2025-02-04 18:07:55|30003045|            6|
|1012-10-0|2025-02-04 18:09:30|2025-02-04 18:09:30|30003044|            7|
|1012-10-0|2025-02-04 18:11:05|2025-02-04 18:11:05|30003043|            8|
|1012-10-0|2025-02-04 18:12:40|2025-02-04 18:12:40|30003013|            9|
|1012-10-0|2025-02-04 18:14:15|2025-02-04 18:14:15|  301693|           10|
+---------+--------------

In [5]:
#shapes

df = spark.read.option("inferSchema", True).option("header", "true") \
    .csv(f"s3a://raw/shapes.txt")
# show data
df.show(10)

df.printSchema()

# Write DataFrame to ClickHouse
df.write \
    .format("jdbc") \
    .option("driver", driver) \
    .option("url", url) \
    .option("user", user) \
    .option("password", password) \
    .option("dbtable", "shapes") \
    .option("createTableOptions", "ENGINE=MergeTree() ORDER BY (shape_id)") \
    .mode("overwrite") \
    .save()

+--------+------------+------------+-----------------+-------------------+
|shape_id|shape_pt_lat|shape_pt_lon|shape_pt_sequence|shape_dist_traveled|
+--------+------------+------------+-----------------+-------------------+
|   81072|  -23.432024|  -46.787121|                1|                0.0|
|   81072|  -23.431827|  -46.787153|                2|          22.061337|
|   81072|  -23.431764|  -46.787163|                3|          29.113285|
|   81072|  -23.431614|  -46.787187|                4|          45.906006|
|   81072|   -23.43159|  -46.787191|                5|          48.596806|
|   81072|  -23.431554|  -46.787196|                6|          52.615639|
|   81072|  -23.431512|  -46.787202|                7|          57.306896|
|   81072|  -23.431438|   -46.78721|                8|          65.543129|
|   81072|  -23.431388|  -46.787215|                9|           71.10347|
|   81072|  -23.431378|  -46.787218|               10|          72.248848|
+--------+------------+--

In [6]:
#routes

df = spark.read.option("inferSchema", True).option("header", "true") \
    .csv(f"s3a://raw/routes.txt")
# show data
df.show(10)

df.printSchema()

# Write DataFrame to ClickHouse
df.write \
    .format("jdbc") \
    .option("driver", driver) \
    .option("url", url) \
    .option("user", user) \
    .option("password", password) \
    .option("dbtable", "routes") \
    .option("createTableOptions", "ENGINE=MergeTree() ORDER BY (route_id)") \
    .mode("overwrite") \
    .save()

+--------+---------+----------------+--------------------+----------+-----------+----------------+
|route_id|agency_id|route_short_name|     route_long_name|route_type|route_color|route_text_color|
+--------+---------+----------------+--------------------+----------+-----------+----------------+
| 1012-10|        1|         1012-10|Term. Jd. Britani...|         3|     509E2F|          FFFFFF|
| 1012-21|        1|         1012-21|Term. Jd. Britâni...|         3|     509E2F|          FFFFFF|
| 1014-10|        1|         1014-10|Morro Doce - Cptm...|         3|     509E2F|          FFFFFF|
| 1015-10|        1|         1015-10|Term. Jd. Britâni...|         3|     509E2F|          FFFFFF|
| 1016-10|        1|         1016-10|Cem. Do Horto - S...|         3|     002F6C|          FFFFFF|
| 1017-10|        1|         1017-10|Perus - Conexão V...|         3|     509E2F|          FFFFFF|
| 1018-10|        1|         1018-10|Vl. Rosa - Metrô ...|         3|     002F6C|          FFFFFF|
| 1019-10|

In [7]:
#frequencies

df = spark.read.option("inferSchema", True).option("header", "true") \
    .csv(f"s3a://raw/frequencies.txt")
# show data
df.show(10)

df.printSchema()

# Write DataFrame to ClickHouse
df.write \
    .format("jdbc") \
    .option("driver", driver) \
    .option("url", url) \
    .option("user", user) \
    .option("password", password) \
    .option("dbtable", "frequencies") \
    .option("createTableOptions", "ENGINE=MergeTree() ORDER BY (trip_id,start_time)") \
    .mode("overwrite") \
    .save()

+---------+-------------------+-------------------+------------+
|  trip_id|         start_time|           end_time|headway_secs|
+---------+-------------------+-------------------+------------+
|1012-10-0|2025-02-04 00:00:00|2025-02-04 00:59:00|        1200|
|1012-10-0|2025-02-04 01:00:00|2025-02-04 01:59:00|        3600|
|1012-10-0|2025-02-04 05:00:00|2025-02-04 05:59:00|         600|
|1012-10-0|2025-02-04 06:00:00|2025-02-04 06:59:00|         600|
|1012-10-0|2025-02-04 07:00:00|2025-02-04 07:59:00|         600|
|1012-10-0|2025-02-04 08:00:00|2025-02-04 08:59:00|         900|
|1012-10-0|2025-02-04 09:00:00|2025-02-04 09:59:00|         900|
|1012-10-0|2025-02-04 10:00:00|2025-02-04 10:59:00|         900|
|1012-10-0|2025-02-04 11:00:00|2025-02-04 11:59:00|         900|
|1012-10-0|2025-02-04 12:00:00|2025-02-04 12:59:00|         900|
+---------+-------------------+-------------------+------------+
only showing top 10 rows

root
 |-- trip_id: string (nullable = true)
 |-- start_time: tim

In [8]:
df = spark.read \
    .format("jdbc") \
    .option("driver", driver) \
    .option("url", url) \
    .option("user", user) \
    .option("password", password) \
    .option("dbtable", "example_table10") \
    .load()

df.show()


Py4JJavaError: An error occurred while calling o151.load.
: java.sql.SQLException: Code: 60. DB::Exception: Unknown table expression identifier 'example_table10' in scope SELECT * FROM example_table10 WHERE 1 = 0. (UNKNOWN_TABLE) (version 25.1.1.4165 (official build))
, server ClickHouseNode(addr=http:clickhouse/<unresolved>:8123, db=cameraaberta)@-1051864132
	at com.clickhouse.jdbc.SqlExceptionUtils.handle(SqlExceptionUtils.java:54)
	at com.clickhouse.jdbc.SqlExceptionUtils.handle(SqlExceptionUtils.java:69)
	at com.clickhouse.jdbc.internal.ClickHouseStatementImpl.executeStatement(ClickHouseStatementImpl.java:139)
	at com.clickhouse.jdbc.internal.SqlBasedPreparedStatement.executeBatch(SqlBasedPreparedStatement.java:158)
	at com.clickhouse.jdbc.internal.SqlBasedPreparedStatement.executeQuery(SqlBasedPreparedStatement.java:199)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCRDD$.getQueryOutputSchema(JDBCRDD.scala:68)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCRDD$.r